# 全景市場分析儀 - Colab 執行器

**指揮官，這是您與「全景市場分析儀」系統互動的主要介面。**

這個 Colab Notebook 遵循「極度簡潔」UI 層原則，主要分為兩個部分：
1.  **環境部署與設定**：自動掛載 Google Drive、從 GitHub 同步最新程式碼、安裝依賴、並導入核心指揮官模組。
2.  **執行分析任務**：您只需要在此處定義要分析的資產、時間範圍等參數，然後向指揮官下達指令即可。

---

In [ ]:
# ===================================================================
# Cell 1: 環境部署與設定
# ===================================================================

# 1.1 掛載 Google Drive (如果數據庫和快取要持久化於此)
# -------------------------------------------------------------------
print("STEP 1.1: Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True) # force_remount 可確保每次都重新掛載
    print("✅ Google Drive mounted successfully at /content/drive.")
except ImportError:
    print("⚠️ Not running in Google Colab environment. Skipping Drive mount.")
except Exception as e:
    print(f"❌ Error mounting Google Drive: {e}")

# 1.2 定義專案路徑與 GitHub 資訊
# -------------------------------------------------------------------
print("\nSTEP 1.2: Defining project paths and GitHub info...")
PROJECT_ROOT_PATH = "/content/panoramic-market-analyzer"
GITHUB_USERNAME = "YOUR_USERNAME"  # <<<<< 指揮官，請在此填寫您的 GitHub 使用者名稱
GITHUB_REPO_NAME = "YOUR_REPO_NAME" # <<<<< 指揮官，請在此填寫您的 GitHub 儲存庫名稱
GITHUB_REPO_URL = f"https://github.com/{GITHUB_USERNAME}/{GITHUB_REPO_NAME}.git"

print(f"Project root will be: {PROJECT_ROOT_PATH}")
print(f"GitHub repository: {GITHUB_REPO_URL}")
print("✅ Project paths and GitHub info defined.")

# 1.3 從 GitHub 同步最新程式碼
# -------------------------------------------------------------------
import os
print("\nSTEP 1.3: Syncing with GitHub repository...")
if os.path.exists(PROJECT_ROOT_PATH):
    print(f"Project directory {PROJECT_ROOT_PATH} already exists. Removing it to ensure fresh clone.")
    # 使用 shell 命令 rm -rf，確保能刪除非空目錄
    !rm -rf {PROJECT_ROOT_PATH}
print(f"Cloning repository {GITHUB_REPO_URL} into {PROJECT_ROOT_PATH}...")
!git clone {GITHUB_REPO_URL} {PROJECT_ROOT_PATH}

# 檢查是否成功 clone
if not os.path.isdir(os.path.join(PROJECT_ROOT_PATH, '.git')):
    print(f"❌ CRITICAL: Failed to clone repository from {GITHUB_REPO_URL}.")
    print("Please check the GITHUB_USERNAME and GITHUB_REPO_NAME variables, and ensure the repository is public or you have access.")
    # 在此處可以選擇停止執行，或拋出錯誤
    # raise RuntimeError("Failed to clone repository.") 
else:
    print("✅ Repository cloned successfully.")
    %cd {PROJECT_ROOT_PATH}
    print(f"Current working directory: {os.getcwd()}")
    print("Listing project root contents:")
    !ls -l

# 1.4 安裝/更新依賴
# -------------------------------------------------------------------
REQUIREMENTS_PATH = os.path.join(PROJECT_ROOT_PATH, "requirements.txt")
print("\nSTEP 1.4: Installing/updating dependencies...")
if os.path.exists(REQUIREMENTS_PATH):
    print(f"Found requirements.txt at {REQUIREMENTS_PATH}. Installing...")
    !pip install --upgrade -q -r {REQUIREMENTS_PATH}
    print("✅ Dependencies installation attempt finished.")
else:
    print(f"⚠️ WARNING: requirements.txt not found at {REQUIREMENTS_PATH}. Skipping dependency installation.")
    print("Ensure your repository contains a requirements.txt file at the root.")

# 1.5 設定 Python 環境，將專案加入 sys.path
# -------------------------------------------------------------------
import sys
print("\nSTEP 1.5: Updating Python sys.path...")
if PROJECT_ROOT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_ROOT_PATH)
    print(f"Added {PROJECT_ROOT_PATH} to sys.path.")
else:
    print(f"{PROJECT_ROOT_PATH} is already in sys.path.")
# print(f"Current sys.path: {sys.path}") # 可選：打印查看 sys.path
print("✅ Python environment configured.")

# 1.6 導入您的核心指揮官及其他必要模組
# -------------------------------------------------------------------
print("\nSTEP 1.6: Importing Commander and other modules...")
try:
    from data_pipeline.commander import Commander
    import pandas as pd # 常用於數據查看
    import logging # 用於設定日誌級別

    # 設定日誌級別，以便在 Colab 中看到 Commander 的詳細輸出
    # logging.basicConfig(level=logging.INFO, 
    #                     format='%(asctime)s - %(levelname)s - [%(name)s] - %(message)s', 
    #                     datefmt='%Y-%m-%d %H:%M:%S',
    #                     force=True) # force=True 在 Colab 中可能需要，以覆蓋預設設定
    # logger = logging.getLogger('data_pipeline') # 獲取 data_pipeline 套件的 logger
    # logger.setLevel(logging.INFO) # 可以設為 DEBUG 以獲得更詳細的輸出
    
    print("🚀 System ready. Commander imported successfully.")
except ImportError as e:
    print(f"❌ CRITICAL ERROR: Failed to import Commander or other modules: {e}")
    print("Please check:")
    print("1. Repository structure: Is 'data_pipeline/commander.py' present?")
    print("2. __init__.py files: Are they present in 'data_pipeline' and its subdirectories?")
    print("3. sys.path: Was the project directory correctly added?")
    print("4. Dependencies: Were all dependencies in requirements.txt installed successfully?")
    print("   (Check for any error messages during the !pip install step above)")
    # raise e # 可以選擇重新拋出錯誤以停止執行
except Exception as e_other:
    print(f"❌ UNEXPECTED ERROR during module import: {e_other}")
    # raise e_other

print("\n🎉🎉🎉 Environment Setup Complete! 🎉🎉🎉")
print("You can now proceed to Cell 2 to execute analysis tasks.")

--- 
## Cell 2: 執行分析任務

**指揮官，這是您與系統互動的唯一地方！**

1.  **設定任務參數**：
    *   `SYMBOLS_TO_ANALYZE`：定義要分析的資產。這是一個字典，鍵為數據源類型 (`equity`, `macro`, `crypto`)，值為對應的符號列表。
    *   `START_DATE` 和 `END_DATE`：定義分析的時間範圍。
    *   `DB_NAME_SUFFIX`：數據庫檔案的後綴，用於區分不同的分析任務或日期。
    *   `FRED_API_KEY` (可選): 如果您的 FRED API 金鑰未設定為環境變數，可以在此處提供。

2.  **實例化指揮官 (Commander)**：傳入必要的路徑和設定。
    *   數據庫和快取檔案將儲存在您的 Google Drive 下的 `MarketData/PanoramicAnalyzer/` 目錄中。

3.  **下達指令給指揮官**：
    *   `run_batch_fetch_and_store`：獲取並儲存指定資產的數據。
    *   `get_processed_data`：獲取已儲存的數據，並可選擇性地進行處理 (例如計算技術指標)。

4.  **查看結果**：獲取的數據將以 Pandas DataFrame 的形式返回，您可以直接在 Notebook 中查看或進一步分析。

---

In [ ]:
# ===================================================================
# Cell 2: 執行分析任務
# ===================================================================
import os
import pandas as pd # 再次導入以確保在此 cell 可用
from datetime import datetime

# --- 1. 設定任務參數 ---
print("STEP 2.1: Defining task parameters...")

# 要分析的資產列表 (按數據源分類)
# 指揮官，請根據您的需求修改此處的資產列表。
# yfinance: 使用 Yahoo Finance 的 Ticker, e.g., 'AAPL', 'SPY', 'BTC-USD'
# fred: 使用 FRED Series ID, e.g., 'DGS10', 'CPIAUCSL', 'VIXCLS'
# coingecko: 使用 CoinGecko Coin ID, e.g., 'bitcoin', 'ethereum', 'solana'
SYMBOLS_TO_ANALYZE = {
    'equity': ['SPY', 'QQQ', 'AAPL', 'NVDA', 'TSLA'],
    'crypto': ['bitcoin', 'ethereum'],
    'macro': ['DGS10', 'VIXCLS'] # 10-Year Treasury, VIX Index
}

START_DATE = "2022-01-01"
END_DATE = datetime.now().strftime('%Y-%m-%d') # 分析到當前日期

# Google Drive 中的數據庫和快取檔案基礎路徑
DRIVE_DATA_BASE_PATH = '/content/drive/MyDrive/MarketData/PanoramicAnalyzer'
# 確保此目錄存在 (如果 Drive 已掛載)
if os.path.exists('/content/drive/MyDrive'):
    os.makedirs(DRIVE_DATA_BASE_PATH, exist_ok=True)
    print(f"Ensured base data directory exists: {DRIVE_DATA_BASE_PATH}")
else:
    print(f"⚠️ Google Drive not mounted or accessible. Data will be stored locally in Colab session: {DRIVE_DATA_BASE_PATH.replace('/content/drive/MyDrive', './local_colab_data')}")
    DRIVE_DATA_BASE_PATH = './local_colab_data/MarketData/PanoramicAnalyzer' # 本地路徑備援
    os.makedirs(DRIVE_DATA_BASE_PATH, exist_ok=True)

# 數據庫檔案名稱 (可以加入日期或任務標識)
DB_NAME_SUFFIX = datetime.now().strftime('%Y%m%d') # 例如，使用當天日期作為後綴
DATABASE_FILE_NAME = f'panoramic_market_data_{DB_NAME_SUFFIX}.duckdb'
DATABASE_PATH = os.path.join(DRIVE_DATA_BASE_PATH, DATABASE_FILE_NAME)

# API 快取檔案名稱
CACHE_FILE_NAME = 'api_requests_cache.sqlite'
CACHE_PATH = os.path.join(DRIVE_DATA_BASE_PATH, CACHE_FILE_NAME)

# FRED API 金鑰 (可選)
# 如果您已將 FRED_API_KEY 設定為 Colab 的 Secrets 或環境變數，則此處可留空 (None)
# 否則，請在此處直接提供您的金鑰字串。
FRED_API_KEY = None # 例如: "your_actual_fred_api_key_here" 
# 嘗試從 Colab Secrets 讀取 (如果存在)
try:
    from google.colab import userdata
    fred_key_secret = userdata.get('FRED_API_KEY') # 假設您在 Colab Secrets 中設定了名為 FRED_API_KEY 的密鑰
    if fred_key_secret:
        FRED_API_KEY = fred_key_secret
        print("Found FRED_API_KEY in Colab Secrets.")
except ImportError:
    pass # 不是在 Colab 環境或 userdata 不可用
except Exception:
    print("No FRED_API_KEY found in Colab Secrets, or error accessing it. Will rely on environment variable or direct input.")

print(f"Task Parameters:")
print(f"  Symbols: {SYMBOLS_TO_ANALYZE}")
print(f"  Start Date: {START_DATE}")
print(f"  End Date: {END_DATE}")
print(f"  Database Path: {DATABASE_PATH}")
print(f"  Cache Path: {CACHE_PATH}")
print(f"  FRED API Key Provided: {'Yes' if FRED_API_KEY else 'No (will try env var or default)'}")
print("✅ Task parameters defined.")

# --- 2. 實例化指揮官 (Commander) ---
print("\nSTEP 2.2: Initializing Commander...")
commander_instance = None # 初始化以備 finally 區塊使用
try:
    # 設定檔 config.yaml 應位於專案根目錄下
    config_file_path = os.path.join(PROJECT_ROOT_PATH, 'config.yaml')
    if not os.path.exists(config_file_path):
        raise FileNotFoundError(f"CRITICAL: config.yaml not found at {config_file_path}. Ensure it's in the project root.")

    # 重新設定日誌 (確保 Commander 的日誌可見)
    import logging
    logging.basicConfig(level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - [%(name)s] - %(message)s', 
                        datefmt='%Y-%m-%d %H:%M:%S',
                        force=True) # force=True 在 Colab 中可能需要
    logging.getLogger('data_pipeline').setLevel(logging.INFO) # 設定 data_pipeline 套件的日誌級別
    logging.getLogger('duckdb').setLevel(logging.WARNING) # DuckDB 的日誌較多，設為 WARNING
    logging.getLogger('yfinance').setLevel(logging.WARNING) # yfinance 的日誌也較多

    commander_instance = Commander(
        config_path=config_file_path,
        db_path=DATABASE_PATH,
        cache_path=CACHE_PATH,
        fred_api_key=FRED_API_KEY
    )
    print("✅ Commander initialized successfully.")
except FileNotFoundError as fnf_err:
    print(fnf_err)
    # 停止執行，因為沒有設定檔，Commander 無法工作
    raise
except Exception as e_init:
    print(f"❌ ERROR: Failed to initialize Commander: {e_init}")
    print("Please check error messages above, ensure config.yaml is correct, and paths are valid.")
    # raise e_init # 可以選擇重新拋出錯誤

# --- 3. 向指揮官下達指令 --- 
if commander_instance:
    try:
        # 3.1 執行批次數據獲取與儲存
        print("\nSTEP 2.3.1: Running batch data fetch and store...")
        db_table_name = 'ohlcv_daily_master' # 主數據表名
        commander_instance.run_batch_fetch_and_store(
            symbols_map=SYMBOLS_TO_ANALYZE,
            start_date=START_DATE,
            end_date=END_DATE,
            table_name=db_table_name
        )
        print(f"✅ Batch data fetch and store process completed. Data should be in table '{db_table_name}' in {DATABASE_PATH}.")

        # 3.2 示例：獲取 SPY 的處理後數據並計算一些指標
        print("\nSTEP 2.3.2: Example - Getting processed data for SPY with indicators...")
        target_symbol_example = 'SPY' # 從 SYMBOLS_TO_ANALYZE 中選一個股票代碼
        if target_symbol_example in SYMBOLS_TO_ANALYZE.get('equity', []):
            indicators_to_calculate = [
                {'name': 'sma', 'window': 20, 'price_col': 'close'},
                {'name': 'sma', 'window': 50, 'price_col': 'close'},
                {'name': 'ema', 'window': 12, 'price_col': 'close'},
                {'name': 'rsi', 'window': 14, 'price_col': 'close'}
            ]
            spy_data_processed = commander_instance.get_processed_data(
                symbol=target_symbol_example,
                table_name=db_table_name,
                start_date=START_DATE,
                end_date=END_DATE,
                indicators=indicators_to_calculate
            )

            if spy_data_processed is not None and not spy_data_processed.empty:
                print(f"\nSuccessfully retrieved and processed data for {target_symbol_example}:")
                print(f"Shape of processed data: {spy_data_processed.shape}")
                print(f"Columns: {spy_data_processed.columns.tolist()}")
                print("Last 5 rows of processed data for SPY:")
                # 使用 display 函數在 Colab 中獲得更好的 DataFrame 輸出
                from IPython.display import display 
                display(spy_data_processed.tail())
            else:
                print(f"⚠️ Could not retrieve or process data for {target_symbol_example}. It might not have been fetched or period is too short.")
        else:
            print(f"⚠️ Example symbol '{target_symbol_example}' not in the 'equity' list to analyze. Skipping processed data example.")

    except Exception as e_runtime:
        print(f"❌ RUNTIME ERROR during Commander operations: {e_runtime}")
        import traceback
        traceback.print_exc() # 打印詳細的錯誤追蹤
    finally:
        # --- 4. 確保資源被釋放 ---
        print("\nSTEP 2.4: Closing Commander to release resources...")
        if commander_instance:
            commander_instance.close()
        print("✅ Commander closed.")
else:
    print("Commander was not initialized. Skipping execution of tasks.")

print("\n🏁🏁🏁 指揮官任務執行完畢！🏁🏁🏁")
print(f"請檢查您的 Google Drive (如果已掛載) 中的數據庫檔案: {DATABASE_PATH}")
print(f"以及快取檔案: {CACHE_PATH}")